# Biome Level statistics for model: Country Level

In [11]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [12]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [13]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [14]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the present forest cover
presentForestCover = compositeImage.select('PresentTreeCover')
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')

# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialForestCover.gt(0)

# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
stdProj = compositeImage.projection();
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Prepare the existing carbon density map: ensemble mean

In [15]:
# load the GS carbon density maps of present biomass
carbonDensityGS_Max = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Present_density_Ensambled_Mean_20230427").unmask()
carbonDensityGS_Mean = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Present_density_Ensambled_Mean_20230427").unmask()
# load the SD carbon density maps of present biomass
carbonDensityWK =  ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km").unmask()
# since the SD map from Arnan is biomass density, therefore, we directly transfer thre biomass density into carbon density by multiplying the carbon concentration
carbonDensitySD =  ee.Image("users/leonidmoore/ForestBiomass/RemoteSensingModel/ESA_CCI_AGB_Map_bias_corrected_1km_2010").multiply(carbonConcentration).unmask()
carbonDensityHM =  ee.Image("users/leonidmoore/ForestBiomass/SpawnMap/Spawn_Harmonized_AGB_density_Map_1km").select('agb').unmask()

## 3 Prepare the potential carbon density map: ensemble mean

In [16]:
# load the potential carbon density for the GS models
potentialDensity_Max1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()
potentialDensity_Max2 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()

potentialDensity_Mean1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()
potentialDensity_Mean2 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()

# load the potential carbon density maps for SD models
potentialDensityWK1 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK1_Potential_density_Ensambled_Mean").unmask()
potentialDensityWK2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK2_Potential_density_Ensambled_Mean").unmask()

potentialDensitySD1 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD1_Potential_density_Ensambled_Mean").unmask()
potentialDensitySD2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD2_Potential_density_Ensambled_Mean").unmask()

potentialDensityHM1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_HM1_Potential_density_Ensambled_Mean").unmask()
potentialDensityHM2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_HM2_Potential_density_Ensambled_Mean").unmask()


In [17]:
# define a functiion to do the values adjustment
def adjFunc(presentDen,potentialDen):
    # check the difference of the two density maps
    potentialHigher = potentialDen.subtract(presentDen).gte(0)
    potentialLower = potentialDen.subtract(presentDen).lt(0)
    # replace the lower potential value by present biomass density value
    adjPotentialDen = presentDen.multiply(potentialLower).add(potentialDen.multiply(potentialHigher))
    return(adjPotentialDen)

# GS max
tgbPotentialDensityGS_Max1 = adjFunc(presentDen = carbonDensityGS_Max,potentialDen = potentialDensity_Max1).multiply(rootShootRatio.add(1))
tgbPotentialDensityGS_Max2 = adjFunc(presentDen = carbonDensityGS_Max,potentialDen = potentialDensity_Max2).multiply(rootShootRatio.add(1))
tgbPresentDensityGS_Max = carbonDensityGS_Max.multiply(rootShootRatio.add(1))
# GS Mean 
tgbPotentialDensityGS_Mean1 = adjFunc(presentDen = carbonDensityGS_Mean,potentialDen = potentialDensity_Mean1).multiply(rootShootRatio.add(1))
tgbPotentialDensityGS_Mean2 = adjFunc(presentDen = carbonDensityGS_Mean,potentialDen = potentialDensity_Mean2).multiply(rootShootRatio.add(1))
tgbPresentDensityGS_Mean = carbonDensityGS_Mean.multiply(rootShootRatio.add(1))
# SD
tgbPotentialDensitySD1 = adjFunc(presentDen = carbonDensitySD,potentialDen = potentialDensitySD1).multiply(rootShootRatio.add(1))
tgbPotentialDensitySD2 = adjFunc(presentDen = carbonDensitySD,potentialDen = potentialDensitySD2).multiply(rootShootRatio.add(1))
tgbPresentDensitySD = carbonDensitySD.multiply(rootShootRatio.add(1))
# HM
tgbPotentialDensityHM1 = adjFunc(presentDen = carbonDensityHM,potentialDen = potentialDensityHM1).multiply(rootShootRatio.add(1))
tgbPotentialDensityHM2 = adjFunc(presentDen = carbonDensityHM,potentialDen = potentialDensityHM2).multiply(rootShootRatio.add(1))
tgbPresentDensityHM = carbonDensityHM.multiply(rootShootRatio.add(1))

# WK
tgbPotentialDensityWK1 = adjFunc(presentDen = carbonDensityWK,potentialDen = potentialDensityWK1).multiply(rootShootRatio.add(1))
tgbPotentialDensityWK2 = adjFunc(presentDen = carbonDensityWK,potentialDen = potentialDensityWK2).multiply(rootShootRatio.add(1))
tgbPresentDensityWK = carbonDensityWK.multiply(rootShootRatio.add(1))

In [18]:
tgbPresentDensityImage = tgbPresentDensityGS_Max.addBands(tgbPresentDensityGS_Mean).addBands(tgbPresentDensitySD).addBands(tgbPresentDensityHM).addBands(tgbPresentDensityWK)
# apply the reducer 
tgbPresentDensity = tgbPresentDensityImage.reduce(ee.Reducer.mean())

# compose the  images into a multiband image
tgbPotentialDensityImage = tgbPotentialDensityGS_Max1.addBands(tgbPotentialDensityGS_Max2).addBands(tgbPotentialDensityGS_Mean1).addBands(tgbPotentialDensityGS_Mean2).addBands(tgbPotentialDensitySD1).addBands(tgbPotentialDensitySD2).addBands(tgbPotentialDensityHM1).addBands(tgbPotentialDensityHM2).addBands(tgbPotentialDensityWK1).addBands(tgbPotentialDensityWK2)
# apply the reducer 
tgbPotentialDensity = tgbPotentialDensityImage.reduce(ee.Reducer.mean())

## 4 Partioning the potential cover into different landuse types

In [19]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap =potentialForestCover.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest potentialForestCover
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [20]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = tgbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = tgbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

### 5.1 Calculate the TGB carbon stock values and write into Google Cloud Storage

In [21]:
# read the country borders polygon
countryBorders = ee.FeatureCollection("users/leonidmoore/ForestBiomass/World_borders_polygon")#.get('ADM0_NAME')

# Stack the absolute biomass layers into an Image.
absPotentialImageTGB = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8)
# define the function to do the country level statistics which could be applied by map      
def countryLevelStat(fc):
    maskedImg = absPotentialImageTGB.clip(fc.geometry())
    output = maskedImg.reduceRegion(reducer= ee.Reducer.sum(),
                                    geometry= unboundedGeo,
                                    crs='EPSG:4326',
                                    crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                    maxPixels= 1e13)
    # generat the eedictionary contains the counrty or region names
    countryRegionName = ee.Feature(fc).get('NAME')
    fcOut = ee.Feature(ee.Geometry.Point([0,0])).set(output).set('Name',countryRegionName)
    return fcOut



statisticTable = countryBorders.map(algorithm = countryLevelStat)
countryStat = ee.batch.Export.table.toCloudStorage(collection = statisticTable, 
                                                   description = 'Country_Level_Statiscis_of_TGB_export',
                                                   bucket = "crowtherlab_gcsb_lidong", #replace it by your own bucket
                                                   fileNamePrefix = 'ForestBiomassExport/Country_Level_Statistics_of_TGB_20230427',
                                                   fileFormat = "CSV")


# start the export task
countryStat.start()
# show the task status
countryStat.status()

{'state': 'READY',
 'description': 'Country_Level_Statiscis_of_TGB_export',
 'creation_timestamp_ms': 1683721807305,
 'update_timestamp_ms': 1683721807305,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': '7F34PPBJGNBPEBPTCBQ5YNRL',
 'name': 'projects/earthengine-legacy/operations/7F34PPBJGNBPEBPTCBQ5YNRL'}

### 5.2 Calculate the PGB carbon stock values and write into Google Cloud Storage

In [22]:
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()

# multiply the dead wood and ratio to get the PGB for each country
absPotentialImagePGB = absPotentialImageTGB.multiply(deadWoodLitterRatio)
# define the function to do the biome level statistics which could be applied by map      
def countryLevelStat(fc):
    maskedImg = absPotentialImagePGB.clip(fc.geometry())
    output = maskedImg.reduceRegion(reducer= ee.Reducer.sum(),
                                    geometry= unboundedGeo,
                                    crs='EPSG:4326',
                                    crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                    maxPixels= 1e13)
    # generat the eedictionary contains the counrty or region names
    countryRegionName = ee.Feature(fc).get('NAME')
    fcOut = ee.Feature(ee.Geometry.Point([0,0])).set(output).set('Name',countryRegionName)
    return fcOut


statisticTable = countryBorders.map(algorithm = countryLevelStat)
countryPGB = ee.batch.Export.table.toCloudStorage(collection = statisticTable, 
                                                  description = 'Country_Level_Statiscis_of_PGB_export',
                                                  bucket = "crowtherlab_gcsb_lidong", #replace it by your own bucket
                                                  fileNamePrefix = 'ForestBiomassExport/Country_Level_Statistics_of_PGB_20230427',
                                                  fileFormat = "CSV")


# start the export task
countryPGB.start()
# show the task status
countryPGB.status()

{'state': 'READY',
 'description': 'Country_Level_Statiscis_of_PGB_export',
 'creation_timestamp_ms': 1683721808537,
 'update_timestamp_ms': 1683721808537,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'PEJDVXVJCYVMXHAYCGGARVSS',
 'name': 'projects/earthengine-legacy/operations/PEJDVXVJCYVMXHAYCGGARVSS'}

## 5.3 Calculate the PGB carbon stock values and write into Google Cloud Storage

In [23]:
# load the carbon density layers
SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# mask the diffrence layer
SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0)).multiply(potentialForestCover.gte(0.1))

# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
# absoluteImage1 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(presentForestCover).divide(1000000000).mask(potentialMask)
absoluteImage2 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(maximumPotentialCover).divide(1000000000).mask(potentialMask).rename('ConservationPotential')
absoluteImage3 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(potentialCoverFinal).divide(1000000000).mask(potentialMask).rename('AbsolutePotential')
absoluteImage4 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(freelandLeftMap).divide(1000000000).mask(potentialMask).rename('FreelandPotential')
absoluteImage5 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(rangelandPotentialCover).divide(1000000000).mask(potentialMask).rename('RangelandPotential')
absoluteImage6 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(pasturePotentialCover).divide(1000000000).mask(potentialMask).rename('PasturePotential')
absoluteImage7 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(croplandPotentialCover).divide(1000000000).mask(potentialMask).rename('CroplandPotential')
absoluteImage8 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(urbanPotentialCover).divide(1000000000).mask(potentialMask).rename('UrbanPotential')
# absoluteImage9 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(freelandForConsevation).divide(1000000000).mask(potentialMask)

In [24]:
# Stack the absolute biomass layers into an Image.
absPotentialImageSoil = absoluteImage2.addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8)
# read the country borders polygon
countryBorders = ee.FeatureCollection("users/leonidmoore/ForestBiomass/World_borders_polygon")#.get('ADM0_NAME')

# define the function to do the country level statistics which could be applied by map      
def countryLevelStat(fc):
    maskedImg = absPotentialImageSoil.clip(fc.geometry())
    output = maskedImg.reduceRegion(reducer= ee.Reducer.sum(),
                                    geometry= unboundedGeo,
                                    crs='EPSG:4326',
                                    crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                    maxPixels= 1e13)
    # generat the eedictionary contains the counrty or region names
    countryRegionName = ee.Feature(fc).get('NAME')
    fcOut = ee.Feature(ee.Geometry.Point([0,0])).set(output).set('Name',countryRegionName)
    return fcOut



statisticTable = countryBorders.map(algorithm = countryLevelStat)
countryStat = ee.batch.Export.table.toCloudStorage(collection = statisticTable, 
                                                   description = 'Country_Level_Statiscis_SoilCarbon_export',
                                                   bucket = "crowtherlab_gcsb_lidong", #replace it by your own bucket
                                                   fileNamePrefix = 'ForestBiomassExport/Country_Level_Statistics_of_SoilCarbon_20230427',
                                                   fileFormat = "CSV")


# start the export task
countryStat.start()
# show the task status
countryStat.status()


{'state': 'READY',
 'description': 'Country_Level_Statiscis_SoilCarbon_export',
 'creation_timestamp_ms': 1683721810018,
 'update_timestamp_ms': 1683721810018,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'ZIBVHQH4WOW5MTTHESM7Z3Y2',
 'name': 'projects/earthengine-legacy/operations/ZIBVHQH4WOW5MTTHESM7Z3Y2'}

In [ ]:
# All those tables were donloaded from cloud storage to local folder 'Data/BiomeLevelStatistics/StatisticsForModels'